# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397876843038                   -0.90    5.2   28.1ms
  2   -8.400236693321       -2.63       -1.74    1.0   20.0ms
  3   -8.400406511497       -3.77       -2.96    1.8   21.6ms
  4   -8.400427864393       -4.67       -2.98    3.2   80.6ms
  5   -8.400427932125       -7.17       -3.02    1.0   20.2ms
  6   -8.400428144727       -6.67       -4.61    1.0   20.2ms
  7   -8.400428151732       -8.15       -4.41    2.8   25.5ms
  8   -8.400428152185       -9.34       -5.27    1.0   20.5ms
  9   -8.400428152207      -10.66       -6.27    1.5   21.6ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397881680250                   -0.90           5.2    908ms
  2   -8.400382653024       -2.60       -1.78   0.80    2.0    565ms
  3   -8.400422585802       -4.40       -3.00   0.80    1.0    235ms
  4   -8.400428111982       -5.26       -3.43   0.80    2.8   23.0ms
  5   -8.400428149229       -7.43       -4.65   0.80    1.0   17.9ms
  6   -8.400428152183       -8.53       -5.97   0.80    2.5   22.2ms
  7   -8.400428152209      -10.59       -5.99   0.80    3.5   24.2ms
  8   -8.400428152209      -13.03       -7.44   0.80    1.0   18.0ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.166070902390                   -1.05    3.86s
  2   -1.566163506390        0.44       -0.64    148ms
  3   -4.360100051169        0.45       -0.35   44.4ms
  4   -6.025988837808        0.22       -0.43    104ms
  5   -7.561785230802        0.19       -0.67   45.2ms
  6   -7.999865456209       -0.36       -1.22   33.1ms
  7   -8.222850476009       -0.65       -1.51   33.1ms
  8   -8.292678141588       -1.16       -1.70   33.1ms
  9   -8.355745574583       -1.20       -2.37   33.3ms
 10   -8.374664484170       -1.72       -2.11   63

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397884401920                   -0.90    5.2   28.4ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427981725                   -1.78    8.89s
  2   -8.400428152209       -6.77       -4.03    3.34s
  3   -8.400428152209      -14.75       -7.83    127ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.2017938046716233e-6
|ρ_newton - ρ_scfv| = 1.4394807918957885e-7
|ρ_newton - ρ_dm|   = 3.471308200028946e-6
